In [1]:
import dash_core_components as dcc
import dash_html_components as html
import dash
import time
import importlib

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
from sklearn.svm import SVC
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
import scipy
import plotly.graph_objects as go
import skopt
import plotly.express as px
from sklearn.manifold import TSNE
import io

In [3]:
%%time
data = pd.read_csv('fashion-mnist-processed.csv')
data = data.sample(n=3000)
data.head()
y = data['label']
X = data[data.columns[1:data.shape[1]]]
X.head()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42)

Wall time: 1.57 s


In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div([
    html.Div(
        id='slider_container',
        children=[
            html.Div(
                id='C_container',
                children=[
                    dcc.Slider(
                        id='my-cost-slider',
                        min=0,
                        max=100,
                        step=0.5,
                        value=10
                    ),
                    html.Div(
                        id='slider-output-container-cost',
                        style={'margin-top': 0.5,
                               'margin-bottom': 50
                               }
                    )
                ]
            ),
            html.Div(
                id='gamma_contaniner',
                children=[
                    dcc.Slider(
                        id='my-gamma-slider',
                        min=0,
                        max=100,
                        step=1,
                        value=10
                    ),
                    html.Div(
                        id='slider-output-container-gamma',
                        style={'margin-top': 0.5,
                               'margin-bottom': 50
                               }
                    )
                ]
            ),
            html.Div(
                id='degree_contaniner',
                children=[
                    dcc.Slider(
                        id='my-degree-slider',
                        min=0,
                        max=100,
                        step=1,
                        value=10
                    ),
                    html.Div(
                        id='slider-output-container-degree',
                        style={'margin-top': 0.5,
                               'margin-bottom': 50
                               }
                    )
                ]
            ),
            html.Div(
                id='kernel_contaniner',
                children=[
                    dcc.Dropdown(
                        id='my-kernel-drop',
                        options=[
                            {'value': x, 'label': x} for x in ['rbf', 'linear', 'poly', 'sigmoid']
                        ],
                        value='linear',
                    ),
                    html.Div(
                        id='slider-output-container-kernel',
                        style={'margin-top': 0.5,
                               'margin-bottom': 50
                               }
                    )
                ]
            )
        ],
        style={'width': '35%', 'display': 'inline-block', 'vertical-align': 'top'}
    ),
    html.Div(children=[
        dcc.Dropdown(
            id='show-all-hyp',
            options=[
                {'value': 'show-all', 'label': 'show all Hyperparameter'},
                {'value': 'show-1/2', 'label': 'show one/two Hyperparameter'}
            ], value='show-1/2',
            style={'margin-top': 0.5,
                   'margin-bottom': 50}
        ),
        html.Div(id='tips',
                 style={'height': 30,
                        'background-color': "#f5f7f6",
                        'margin-top': 0.5,
                        'margin-bottom': 50
                        },
                 ),
        dcc.Dropdown(id='Choosen-hyperparameter',
                     options=[
                         {'value': x, 'label': x} for x in ['C', 'gamma', 'kernel', 'degree']
                     ],
                     multi=True,
                     value=['gamma', 'C'],
                     style={'margin-top': 0.5,
                            'margin-bottom': 50}),
        html.Div(
            id="div-graphs",
            children=[
                dcc.Graph(
                    id="graph-hyperparameter-svm",
                    figure=dict(
                        layout=dict(
                            plot_bgcolor="#68786f", paper_bgcolor="#68786f"
                        )  # Sets the background color of the plotting area in- between x and y axes.
                        # Sets the legend background color
                    ),
                )
            ]
        )
    ],
        style={'width': '65%', 'display': 'inline-block', 'vertical-align': 'top'}

    )
]
)


# choosen_hyp axis of scatter

In [ ]:
@app.callback([dash.dependencies.Output('my-cost-slider', 'disabled'),
               dash.dependencies.Output('my-gamma-slider', 'disabled'),
               dash.dependencies.Output('my-kernel-drop', 'disabled'),
               dash.dependencies.Output('my-degree-slider', 'disabled'),
               dash.dependencies.Output('Choosen-hyperparameter', 'disabled')],
              [dash.dependencies.Input('Choosen-hyperparameter', 'value'),
               dash.dependencies.Input('show-all-hyp', 'value')])
def display_slider(choosen_hyp, show):
    hyp_list = ['C', 'gamma', 'kernel', 'degree']
    n_hyp = len(choosen_hyp)
    if show == 'show-1/2':
        for x in range(len(hyp_list)):
            if hyp_list[x] in choosen_hyp:
                hyp_list[x] = True
            else:
                hyp_list[x] = False
        hyp_list.append(False)
    else:
        hyp_list = [True, True, True, True, True]
    return hyp_list


@app.callback(
    dash.dependencies.Output('tips', 'children'),
    [dash.dependencies.Input('Choosen-hyperparameter', 'value')]
)
def Tips_hyp(hyp_list):
    if len(hyp_list) == 0:
        output_tips = 'Error! Please choose at least one hyperparameter'
    else:
        output_tips = 'choose the parameter you want show with scatter plot'

    return [html.Div([html.H5(children=output_tips,
                              style={
                                  'color': '#2c7d66',
                                  'text-align': 'center'
                              }),
                      html.Hr()
                      ]
                     )
            ]


#

@app.callback(
    dash.dependencies.Output('Choosen-hyperparameter', 'options'),
    [dash.dependencies.Input('Choosen-hyperparameter', 'value'),
     dash.dependencies.State('Choosen-hyperparameter', 'options')])
def choose_hyp_disabled(hyp_list, options):
    if len(hyp_list) >= 2:
        for i in range(4):
            options[i]['disabled'] = True
    else:
        for i in range(4):
            options[i]['disabled'] = False
    return options


#

@app.callback(
    dash.dependencies.Output('slider-output-container-cost', 'children'),
    [dash.dependencies.Input('my-cost-slider', 'value')])
def update_output_cost(value):
    return 'You have selected C"{}"'.format(value)


@app.callback(
    dash.dependencies.Output('slider-output-container-gamma', 'children'),
    [dash.dependencies.Input('my-gamma-slider', 'value')])
def update_output_gamma(value):
    return 'You have selected gamma"{}"'.format(value)


@app.callback(
    dash.dependencies.Output('slider-output-container-degree', 'children'),
    [dash.dependencies.Input('my-degree-slider', 'value')])
def update_output_degree(value):
    return 'You have selected degree"{}"'.format(value)


@app.callback(
    dash.dependencies.Output('slider-output-container-kernel', 'children'),
    [dash.dependencies.Input('my-kernel-drop', 'value')])
def update_output_kernel(value):
    return 'You have selected kernel"{}"'.format(value)

# choosen_hyp axis of scatter


@app.callback(
    dash.dependencies.Output('div-graphs', 'children'),
    [dash.dependencies.Input('my-cost-slider', 'value'),
     dash.dependencies.Input('my-gamma-slider', 'value'),
     dash.dependencies.Input('my-kernel-drop', 'value'),
     dash.dependencies.Input('my-degree-slider', 'value'),
     dash.dependencies.Input('Choosen-hyperparameter', 'value'),
     dash.dependencies.Input('show-all-hyp', 'value')
     ]
)
def update_output(C_value, gamma_value, kernel_value, degree_value, choosen_hyp, show):
    data = []
    support = SVC()

    if show == 'show-1/2':
        hyp_list = ['C', 'gamma', 'kernel', 'degree']
        if 'C' in choosen_hyp:
            C = scipy.stats.uniform(scale=100)
        else:
            C = C_value
        if 'gamma' in choosen_hyp:
            gamma = scipy.stats.uniform(scale=1)
        else:
            gamma = gamma_value
        if 'kernel' in choosen_hyp:
            kernel = ['rbf', 'linear', 'poly', 'sigmoid']
        else:
            kernel = kernel_value
        if 'degree' in choosen_hyp:
            degree = scipy.stats.uniform(scale=10)
        else:
            degree = degree_value
        distributions = dict(C=([C] if 'C' not in choosen_hyp else C),
                             gamma=(
                                 [gamma] if 'gamma' not in choosen_hyp else gamma),
                             kernel=(
                             [kernel] if 'kernel' not in choosen_hyp else kernel),
                             degree=([degree] if 'degree' not in choosen_hyp else degree))

        clf = RandomizedSearchCV(support, distributions,
                                 random_state=42, n_iter=50)
        search = clf.fit(X_train, y_train)
        search1 = search.score(X_test, y_test)
        results = search.cv_results_['params']
        results_mean_score = search.cv_results_['mean_test_score']
        results_df = pd.DataFrame(
            results, columns=['C', 'gamma', 'kernel', 'degree'])
        results_df['mean_test_score'] = results_mean_score.tolist()
        text_list = list(
            zip(
                'C: ' + results_df['C'].round(4).apply(str),
                'gamma: ' + results_df['gamma'].round(4).apply(str),
                'kernel: ' + results_df['kernel'].apply(str),
                'degree: ' + results_df['degree'].round(4).apply(str),
                'mean_test_score: ' +
                results_df['mean_test_score'].round(4).apply(str)
            )
        )
        text = ['<br>'.join(t) for t in text_list]

        results_df['Text'] = text
        if len(choosen_hyp) == 1:
            scatter = go.Scatter(
                x=results_df[choosen_hyp[0]],
                y=results_df['mean_test_score'],
                text=results_df['Text'],
                hoverinfo='text',
                textposition="middle center",
                showlegend=False,
                mode="markers",
                marker=dict(size=7,
                            opacity=0.9,
                            line=dict(color='rgb(140, 140, 170)'),
                            color=results_df.mean_test_score,
                            colorscale='Viridis',
                            colorbar=dict(title='Test score')
                            )
            )
            layout = go.Layout(
                title='2D visualization of the random search results',
                margin=dict(
                    l=30,
                    r=30,
                    b=30,
                    t=30
                ),
                #     height=600,
                #     width=960,
                scene=dict(
                    xaxis=dict(
                        title=choosen_hyp[0],
                        nticks=10
                    ),
                    yaxis=dict(
                        title='mean_test_score',
                    )
                ),
            )
        elif len(choosen_hyp) == 2:
            scatter = go.Scatter3d(
                x=results_df[choosen_hyp[0]],
                y=results_df[choosen_hyp[1]],
                z=results_df['mean_test_score'],
                text=results_df['Text'],
                hoverinfo='text',
                textposition="middle center",
                showlegend=False,
                mode="markers",
                marker=dict(size=6,
                            opacity=0.9,
                            line=dict(color='rgb(140, 140, 170)'),
                            color=results_df.mean_test_score,
                            colorscale='Viridis',
                            colorbar=dict(title='Test score')
                            )
            )
            layout = go.Layout(
                title='3D visualization of the random search results',
                margin=dict(
                    l=30,
                    r=30,
                    b=30,
                    t=30
                ),
                #     height=600,
                #     width=960,
                scene=dict(
                    xaxis=dict(
                        title=choosen_hyp[0],
                        nticks=10
                    ),
                    yaxis=dict(
                        title=choosen_hyp[1],
                    ),
                    zaxis=dict(
                        title='mean_test_score',
                    ),
                ),
            )
            data.append(scatter)
            prediction_figure = go.Figure(data=data, layout=layout)
        return [
            html.Div(
                id="svm-graph-container",
                children=dcc.Loading(
                    className="graph-wrapper",
                    children=dcc.Graph(id="graph-hyperparameter-svm",
                                       figure=prediction_figure),
                    style={"display": "none"},
                ),
            )
        ]

    elif show == 'show-all':
        distributions = dict(C=scipy.stats.uniform(scale=100), gamma=scipy.stats.uniform(scale=.1),
                             kernel=['rbf', 'linear', 'poly', 'sigmoid'], degree=scipy.stats.uniform(scale=10))
        clf = RandomizedSearchCV(
            support, distributions, random_state=0, n_iter=100)
        search = clf.fit(X_train, y_train)
        results = search.cv_results_['params']
        results_mean_score = search.cv_results_['mean_test_score']
        results_df = pd.DataFrame(
            results, columns=['C', 'gamma', 'kernel', 'degree'])
        results_df['mean_test_score'] = results_mean_score.tolist()
        text_list = list(
            zip(
                'C: ' + results_df['C'].round(4).apply(str),
                'gamma: ' + results_df['gamma'].round(4).apply(str),
                'kernel: ' + results_df['kernel'].apply(str),
                'degree: ' + results_df['degree'].round(4).apply(str),
                'mean_test_score: ' +
                results_df['mean_test_score'].round(4).apply(str)
            )
        )
        text = ['<br>'.join(t) for t in text_list]
        results_df['Text'] = text

        results_df['kernel'] = results_df['kernel'].map(
            {'rbf': 0, 'linear': 1, 'poly': 2, 'sigmoid': 3})
        results_df1 = results_df.drop('Text', axis=1)
        results_df2 = results_df1.drop('mean_test_score', axis=1)
        tsne = TSNE(n_components=3, verbose=True, perplexity=40.0)
        embedding = tsne.fit_transform(results_df2)
        embedding_df = pd.DataFrame(embedding, columns=["x", "y", "z"])
        scatter = go.Scatter3d(
            name=str(embedding_df.index),
            x=embedding_df["x"],
            y=embedding_df["y"],
            z=embedding_df["z"],
            text=results_df['Text'],
            hoverinfo='text',
            textposition="middle center",
            showlegend=False,
            mode="markers",
            marker=dict(size=5, symbol="circle",
                        color=results_df['mean_test_score'],
                        colorscale='aggrnyl',
                        colorbar=dict(title=str('mean_test_score'))
                        )
        )
        layout = go.Layout(
            title='3D visualization of the random search results of all parameter',
            margin=dict(
                l=30,
                r=30,
                b=30,
                t=30
            )
        )
        data.append(scatter)
        prediction_figure_all = go.Figure(data=data, layout=layout)
        prediction_figure_parallel = px.parallel_coordinates(results_df, color='mean_test_score',
                                                             dimensions=['C', 'gamma', 'degree',
                                                                         'kernel'],
                                                             color_continuous_scale=px.colors.diverging.Tealrose,
                                                             color_continuous_midpoint=0.500)

        return [
            html.Div(
                id="svm-graph-container",
                children=dcc.Loading(
                    children=[dcc.Graph(id="graph-hyperparameter-svm",
                                           className="graph-wrapper",
                                           figure=prediction_figure_all),
                              dcc.Graph(id="parallel-axis-svm",
                                           figure=prediction_figure_parallel)]
                ),
            )
        ]


if __name__ == '__main__':
    app.run_server(debug=False,port=8050)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Jun/2021 11:31:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:31:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:32:27] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [08/Jun/2021 11:34:18] "POST /_dash-update-component

[t-SNE] Computing 99 nearest neighbors...
[t-SNE] Indexed 100 samples in 0.000s...
[t-SNE] Computed neighbors for 100 samples in 0.018s...
[t-SNE] Computed conditional probabilities for sample 100 / 100
[t-SNE] Mean sigma: 15.904779
[t-SNE] KL divergence after 250 iterations with early exaggeration: 84.251053


127.0.0.1 - - [08/Jun/2021 11:43:14] "POST /_dash-update-component HTTP/1.1" 200 -


[t-SNE] KL divergence after 1000 iterations: 1.240189
